In [1]:
import pandas as pd
import warnings
import nlpaug.augmenter.word as naw


# Mengabaikan peringatan yang muncul
warnings.simplefilter("ignore")

# Mengatur opsi pandas untuk menampilkan maksimal 500 kolom dalam output
pd.set_option("display.max_columns", 500)

# Mengatur lebar tampilan output menjadi 1000 karakter
pd.set_option("display.width", 1000)


g:\CodeWarehouse\project-skripsi\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Membuat fungsi untuk melakukan augmentasi teks
def augment_text(text):
    # Membuat objek augmenter menggunakan model BERT bahasa Indonesia
    augmenter = naw.ContextualWordEmbsAug(
        model_path="cahya/bert-base-indonesian-522M", action="insert"
    )

    # Melakukan augmentasi teks
    augmented_text = augmenter.augment(text)

    # Mengembalikan teks yang telah diaugmentasi
    return augmented_text


data = {
    "Text": [
        "saya tidak suka dengan kebijakan ini",
        "saya senang melihat semangat presiden jokowi dalam bekerja",
    ]
}
df = pd.DataFrame(data)

# Mengaplikasikan augmentasi teks pada DataFrame
df["Augmented_Text"] = df["Text"].apply(augment_text)

# Menampilkan hasil
df

In [3]:
# Membaca file dataset dengan menggunakan pandas
df = pd.read_csv("../dataset/INA_TweetsPPKM_Labeled_PosNeg.csv", sep="\t")

# Menampilkan 5 baris pertama dari dataset
df.head()


,Date,User,Tweet,sentiment
0,2022-03-26 04:51:44+00:00,CendekiaDelapan,Karena stimulasi untuk #anakberkebutuhankhusus...,0
1,2022-03-25 10:13:26+00:00,AnantaElvano,Hadir Kembali Boss!! MEGA WIN Dari https://t.c...,1
2,2022-03-23 23:15:45+00:00,mitrafmpwt,Hayolohhhhhhhhh... seneng kannnn??\n\n#covid19...,1
3,2022-03-23 10:29:52+00:00,895jizfm,Thank You teman-teman dari #saganheritagehotel...,1
4,2022-03-22 17:25:29+00:00,ViantAntony,Ruwet Ruwet Ruwet inilah Negeri RuwetNesia. He...,1


In [4]:
df.columns = df.columns.str.lower()
df.drop(["date", "user"], axis=1, inplace=True)


In [5]:
df_pos = df.loc[df['sentiment']!=0]
df_pos.sample(5)

,tweet,sentiment
1735,Repost @bnn_cegahnarkoba \nSiapa aja #sobatceg...,1
847,Happy Weekend BFFs! Berhubung #PPKM diperpanja...,1
4521,Tetap patuhi prokes ya Friends!!! \n\n#togethe...,1
304,Hi Parents!Ada kabar gembira mulai tanggal 5 O...,1
3453,"PPKM versi aku nih, untung pake smartfren GOKI...",1


In [6]:
%%timeit
df_pos['augmented_text'] = df['tweet'].apply(augment_text)
df.sample(5)

In [ ]:
df_pos.head()

,tweet,sentiment,augmented_text
5232,@ELFARAFM @reviczilia @ngonoafer Bysany nonton...,1,[@ elfarafm @ reviczilia @ go ngonoafer my bys...
5803,"- DTW Umbul Sidomukti, Bandungan.\n- DTW Horti...",1,"[- sesama dtw dukuh umbul sidomukti, bandungan..."
2851,PPKM trus sampe Levi jadi jodohku. \n#PPKM #PP...,1,[ppkm 60 trus levi sampe levi bukan jadi wasia...
1426,"TKA masuk untuk bekerja,dan kita terpuruk kare...",1,"[tka masuk untuk turut bekerja, dan kita mulai..."
542,Yessss sun.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n#ta...,1,[from yessss by sun........... # tattoo # tatt...
2730,"Yakali disamain mah lucu, jauuhh #emyu #Liverp...",1,"[ade yakali mah disamain mah yang lucu, jauuhh..."
2967,Hallo... Ini #PPKM versiku\nga bakalan bosenin...,1,[e hallo... ini # ppkm versiku ga ho bakalan b...
1369,Ini ppkm versi aku untung pakai smartfren goki...,1,[ini ppkm versi reguler aku untung pakai smart...
4633,Masih harus bertahan seadanya. Berjuang utk le...,1,[masih gigih harus bertahan lebih seadanya. be...
2729,PPKM versi aku nih https://t.co/kAoMZ6YX1n\nMa...,1,[ppkm di versi aku nih https : / / t. en co / ...


In [111]:
# Mengganti karakter [ dan ] dengan string kosong
df["Augmented_Text"] = df["Augmented_Text"].str.replace("[", "").str.replace("]", "")

# Mengganti karakter ' dengan string kosong
df["Augmented_Text"] = df["Augmented_Text"].str.replace("'", "")

# Menampilkan informasi tentang DataFrame
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Text            2 non-null      object
 1   Augmented_Text  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [112]:
df.head()

,Text,Augmented_Text
0,saya tidak suka dengan kebijakan ini,jika saya tidak suka dengan kebijakan semacam ini
1,saya senang melihat semangat presiden jokowi d...,bisa saya senang melihat betapa semangat presi...


In [113]:
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# Mengunduh resource yang diperlukan oleh NLTK (Natural Language Toolkit)
nltk.download("punkt")


[nltk_data] Downloading package punkt to C:\Users\SNOW-
[nltk_data]     WOLF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [114]:
def remove_special_char(text):
    # Menghapus karakter khusus seperti tab, baris baru, dan backslash
    text = (
        text.replace("\\t", " ")
        .replace("\\n", " ")
        .replace("\\u", " ")
        .replace("\\", "")
    )
    # Menghapus karakter non-ASCII seperti emotikon
    text = text.encode("ASCII", "replace").decode("ASCII")
    # Menghapus mention, link, dan hashtag
    text = " ".join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())
    # Menghapus URL yang tidak lengkap
    return text.replace("http://", " ").replace("https://", " ")


def remove_number(text):
    # Menghapus angka dari teks
    return re.sub(r"\d+", "", text)


def remove_punctuation(text):
    # Menghapus tanda baca dari teks
    return text.translate(str.maketrans("", "", string.punctuation))


def remove_whitespace(text):
    # Menghapus spasi di awal dan akhir teks
    return text.strip()


def remove_multiple_whitespace(text):
    # Menghapus multiple whitespace menjadi satu spasi
    return re.sub("\s+", " ", text)


def remove_single_char(text):
    # Menghapus kata yang terdiri dari satu karakter saja
    return re.sub(r"\b[a-zA-Z]\b", "", text)


def tokenize(text):
    # Melakukan tokenisasi kata pada teks menggunakan NLTK
    return word_tokenize(text)


In [115]:
# Mengaplikasikan fungsi remove_special_char pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_special_char)

# Mengaplikasikan fungsi remove_number pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_number)

# Mengaplikasikan fungsi remove_punctuation pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_punctuation)

# Mengaplikasikan fungsi remove_whitespace pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_whitespace)

# Mengaplikasikan fungsi remove_multiple_whitespace pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_multiple_whitespace)

# Mengaplikasikan fungsi remove_single_char pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(remove_single_char)

# Mengaplikasikan fungsi tokenize pada kolom 'Augmented_Text'
df["Augmented_Text"] = df["Augmented_Text"].map(tokenize)

# Menampilkan lima baris pertama dari DataFrame setelah dilakukan preprocessing teks
df.head()


,Text,Augmented_Text
0,saya tidak suka dengan kebijakan ini,"[jika, saya, tidak, suka, dengan, kebijakan, s..."
1,saya senang melihat semangat presiden jokowi d...,"[bisa, saya, senang, melihat, betapa, semangat..."
